In [ ]:
import warnings
warnings.filterwarnings("ignore")
from random import seed
from random import randrange
from csv import reader
from math import sqrt
from sklearn import preprocessing
import time
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def log_loss(p, y):
    return (-y * np.log(p) - (1 - y) * np.log(1 - p))

def grad_log_loss(p, y, x):
    return (p - y) * x

def mod_log_loss(p, y_priv, p_RR):
    p_mod = p_RR *  p + (1 - p_RR) * (1 - p) 
    return log_loss(p_mod, y_priv)

def grad_mod_log_loss(p, y_priv, p_RR, x):
    p_mod = p_RR *  p + (1 - p_RR) * (1 - p) 
    return (p * (1 - p) * (1 - 2 * p_RR) / (p_mod - (1 - y_priv))) * x

def priv_log_loss(p, y_priv, p_RR):
    p1 = p_RR *  np.log(p) - (1 - p_RR) * np.log(1 - p) 
    p0 = p_RR *  np.log(1 - p) - (1 - p_RR) * np.log(p) 
    return (- y_priv * p1 - (1 - y_priv) * p0)   

def grad_priv_log_loss(p, y_priv, p_RR, x):
    return (p_RR * (p - y_priv) - (1 - p_RR) * (p - (1 - y_priv))) * x

def update_weight(weight, learning_rate, gradient, maxNorm):
    weightUpd = weight - learning_rate * gradient
    weightProj = weightUpd/np.maximum(1, np.linalg.norm(weightUpd, 2)/maxNorm)
    return weightProj

In [ ]:
start_time = time.time()
d = 5
thetaStar = np.random.randn(d)
B = np.linalg.norm(thetaStar, 2)
Epsilon = np.array([0.1, 0.5, 1], dtype=float)
delta = 0.001
N = np.array([1000, 2000, 4000, 6000, 8000, 10000], dtype=int)
num_iter = 100
thetaInit = np.zeros(d)
lamda = 0.01
SigmaInit = lamda * np.eye(d)
etaInit = 1

errorL2 = np.zeros([len(Epsilon), len(N), num_iter])
errorL2_priv = np.zeros([len(Epsilon), len(N), num_iter])
errorL2_obj = np.zeros([len(Epsilon), len(N), num_iter])
for e in range(len(Epsilon)):
    eps = Epsilon[e]
    for n in range(len(N)):
        num_data = N[n]
        for iter in range(num_iter): 
            X = np.zeros([num_data, d])
            Y = np.zeros(num_data)
            pStar = np.zeros(num_data)
            for i in range(num_data):
                Phi1 = np.random.randn(d)
                Phi0 = np.random.randn(d)
                X[i,:] = Phi1 - Phi0
                pStar[i] = sigmoid(np.dot(thetaStar, X[i,:]))
                Y[i] = np.random.binomial(1, pStar[i])
            L = np.amax(np.linalg.norm(X, axis = 1))
            sigma = L * np.sqrt(np.log(1/delta))/(eps*np.sqrt(num_data))
            Sigma = SigmaInit

            theta = thetaInit
            theta_priv = thetaInit
            theta_obj = thetaInit
            for i in range(num_data):
                eta = etaInit
                x = X[i,:]
                Sigma = Sigma + np.outer(x, x)
                y = Y[i]
                p_RR = sigmoid(eps)
                toss = np.random.binomial(1, p_RR)
                if toss == 1:
                    y_priv = y
                else:
                    y_priv = 1 - y
                p = sigmoid(np.dot(x, theta))
                grad = grad_log_loss(p, y, x)
                theta = update_weight(theta, eta, grad, B)    
                
                p_priv = sigmoid(np.dot(x, theta_priv))
                grad_priv = grad_priv_log_loss(p_priv, y_priv, p_RR, x)
                theta_priv = update_weight(theta_priv, eta, grad_priv, B)

                p_obj = sigmoid(np.dot(x, theta_obj))
                grad_obj = grad_log_loss(p_obj, y, x) + np.random.normal(0, sigma, d)
                theta_obj = update_weight(theta_obj, eta, grad_obj, B)

            errorL2[e, n, iter] = np.linalg.norm(theta - thetaStar, 2) / np.sqrt(num_data)
            errorL2_priv[e, n, iter] = np.linalg.norm(theta_priv - thetaStar, 2) / np.sqrt(num_data)
            errorL2_obj[e, n, iter] = np.linalg.norm(theta_obj - thetaStar, 2) / np.sqrt(num_data)

            print("Training time:" + str(time.time() - start_time) + " seconds")
            print("Learning rate: {}\nIteration: {}".format(0.01, iter))

errorL2_avg = np.mean(errorL2,axis=2)
errorL2_priv_avg = np.mean(errorL2_priv,axis=2)
errorL2_obj_avg = np.mean(errorL2_obj,axis=2)

errorL2_std = np.std(errorL2,axis=2)
errorL2_priv_std = np.std(errorL2_priv,axis=2)
errorL2_obj_std = np.std(errorL2_obj,axis=2)

In [ ]:
plot_dir = os.path.join(".", "plots") # Directory to save plots
os.makedirs(plot_dir, exist_ok=True)

for e in range(len(Epsilon)):
    plt.figure(figsize=(5,3))
    plt.errorbar(N, errorL2_avg[e,:], errorL2_std[e,:], label='MLE (Non-private)')
    plt.errorbar(N, errorL2_priv_avg[e,:], errorL2_priv_std[e,:], label='RR (Local model)')
    plt.errorbar(N, errorL2_obj_avg[e,:], errorL2_obj_std[e,:], label='Obj (Central model)')

    plt.xlabel("Number of Samples")
    plt.ylabel("Estimation Error in L2-norm") 
    plt.legend(loc= 'best', fontsize=12)
    plt.grid(True)

    eps = Epsilon[e]
    fig_name = "L2_norm_" + str(eps) + ".pdf"
    fname = os.path.join(plot_dir, fig_name)
    plt.savefig(fname, format = "pdf", dpi = 1200, bbox_inches="tight")

    print('Error of MLE :', errorL2_avg[e,:])
    print('Error of RR :', errorL2_priv_avg[e,:])
    print('Error of Obj :', errorL2_obj_avg[e,:])